In [1]:
# Import necessary modules
import requests
from bs4 import BeautifulSoup
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from dotenv import load_dotenv
import os
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle

from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.schema import Document

import os
from dotenv import load_dotenv, find_dotenv
from langchain_community.llms import HuggingFaceEndpoint, HuggingFaceHub
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from huggingface_hub import HfApi

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA

from transformers import AutoModel, AutoTokenizer
from langchain.document_loaders import WebBaseLoader
from langchain_core.documents.base import Document
from langchain.vectorstores import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from huggingface_hub import HfApi, HfFolder
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

c:\Users\user\.conda\envs\mllrag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
urls=[
"https://weclouddata.com/courses/online/business-intelligence-bootcamp",
"https://weclouddata.com/course-package/business-intelligence-bootcamp-ca/",
"https://weclouddata.com/course-package/business-intelligence-bootcamp-us/",
"https://weclouddata.com/courses/online/data-science-bootcamp-part-time",
"https://weclouddata.com/course-package/data-science-bootcamp-pt-ca/",
"https://weclouddata.com/course-package/data-science-bootcamp-pt-us/",
"https://weclouddata.com/courses/online/data-science-bootcamp/",
"https://weclouddata.com/course-package/data-science-bootcamp-ft-ca/",
"https://weclouddata.com/course-package/data-science-bootcamp-ft-us/",
"https://weclouddata.com/courses/online/data-engineering-bootcamp-part-time/",
"https://weclouddata.com/course-package/data-engineering-bootcamp-ca/",
"https://weclouddata.com/course-package/data-engineering-bootcamp-us/",
"https://weclouddata.com/courses/online/ml-engineering-bootcamp/",
"https://weclouddata.com/course-package/ml-engineering-bootcamp-ca/",
"https://weclouddata.com/course-package/ml-engineering-bootcamp-us/",
"https://weclouddata.com/courses/us-online/business-intelligence-bootcamp-self-paced/",
"https://weclouddata.com/courses/us-online/data-science-bootcamp-self-paced/",
"https://weclouddata.com/courses/us-online/generative-ai-bootcamp-self-paced/",
"https://weclouddata.com/courses/us-online/data-engineering-bootcamp-self-paced/",
"https://weclouddata.com/courses/us-online/devops-bootcamp-self-paced/",
"https://weclouddata.com/courses/us-online/ml-engineering-bootcamp-self-paced/",
"https://weclouddata.com/courses/us-online/data-fundamentals/",
"https://weclouddata.com/courses/us-online/data-analytics-real-project-course/",
"https://weclouddata.com/courses/us-online/data-science-with-python/",
"https://weclouddata.com/courses/us-online/applied-machine-learning/",
"https://weclouddata.com/courses/us-online/big-data-for-data-scientist/",
"https://weclouddata.com/courses/us-online/data-science-real-project-course/",
"https://weclouddata.com/courses/us-online/analytics-engineering-course/",
"https://weclouddata.com/courses/us-online/big-data-engineering/",
"https://weclouddata.com/courses/us-online/data-engineer-real-project-course/",
"https://weclouddata.com/courses/us-online/ai-course-computer-vision/",
"https://weclouddata.com/courses/us-online/ai-course-nlp-llm/",
"https://weclouddata.com/courses/us-online/mlops-engineering/",
"https://weclouddata.com/courses/us-online/data-analytics-real-project-course/",
"https://weclouddata.com/courses/us-online/data-science-real-project-course/",
"https://weclouddata.com/courses/us-online/data-engineer-real-project-course/"

]

In [3]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv('TokHap.env')
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

# Initialize the embedding model and spaCy NLP model
#embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Set up the LLM from Hugging Face
repo_id = 'mistralai/Mistral-7B-Instruct-v0.2'
mistral_llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_new_tokens": 500})

c:\Users\user\.conda\envs\mllrag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [4]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
def load_clean_text_from_url(url):
  # Send a GET request to the webpage
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      html_content = response.content

      # Parse the HTML content
      soup = BeautifulSoup(html_content, 'html.parser')
      # Find all body elements
      containers = soup.find_all('body')
      # Extract raw text for all body of Html
      for container in containers:
         raw_text = container.get_text(separator=' ', strip=True)
      # Clean the extracted text
      if raw_text:
         clean_text = ' '.join(raw_text.split())
         #print(clean_text)

  else:
     print("Failed to retrieve the webpage")

  return clean_text

In [5]:
import pandas as pd

class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

    def __repr__(self):
        return f"Document(page_content={self.page_content!r}, metadata={self.metadata!r})"


def is_word_at_start(text, word):
    # Strip any leading whitespace from the text and check if it starts with the word
    return text.strip().startswith(word)


def split_text_by_headers(text, headers):
    sections = []
    last_pos = 0
    for i, keyword in enumerate(headers):
        pos = text.find(keyword, last_pos)
        if pos != -1:
            if i > 0:
                sections.append(f" {text[last_pos:pos].strip()}")
            last_pos = pos
    sections.append(f" {text[last_pos:].strip()}")
    
    return sections


def load_headers_creat_metadata(url):

    data = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    page_title = soup.title.string if soup.title else 'No title found'
    path_segment = url.split(".com")[1]

    # Extracting all headers and content
    headers = soup.find_all(['h1', 'h2', 'h3', 'h4','h5'])
    
    for header in headers:
        header_text = header.get_text(strip=True)
        next_sibling = header.find_next_sibling()
        content = ''
        while next_sibling and next_sibling.name not in ['h1', 'h2', 'h3', 'h4','h5']:
            if next_sibling.name:
                content += next_sibling.get_text(strip=True) + " "
            next_sibling = next_sibling.find_next_sibling()
        
        data.append({
            'url': url,
            'path_segment': path_segment,
            'page_title': page_title,
            'header': header_text    
        })
    # Convert to DataFrame and display
    df = pd.DataFrame(data)
    
    text= load_clean_text_from_url(url)
    headers = df['header'].tolist()
    #print('h=',headers)
    #print("<<<<---------->>>>")

    result = split_text_by_headers(text, headers)
    #print('result_sec_num=',type(result),len(result))

    metadata_documents = []
    for i, section in enumerate(result):
        if is_word_at_start(section, headers[i]):
                       
           section_extention = f" >>>>>>\n URL :: {url} \n------\n Title :: {page_title} \n-------\n Link :: {path_segment} \n------\n Section {i} :: {headers[i]} >>>>>>>>>\n content :::: {section} "
           headersec = headers[i]
        else:
           section_extention = f" >>>>>>\n URL :: {url} \n------\n Title :: {page_title} \n-------\n Link :: {path_segment} \n------\n Section {i} :: {headers[i+1]} >>>>>>>>\n content :::: {section} "
           headersec = headers[i+1]
    #print(section_extention)
    #print(">>>>>>>>>\n")
    
        metadata_documents.append(Document(
                    #page_content = section,
                    page_content = section_extention,
                    #page_content=f"Title :: {page_title}, Link :: {path_segment}, Section {i} :: {headers[i]} >>> {section} ",
                    metadata={"source": url,"Title": page_title, "Header": headersec, "Section number" :i }
                    ))
    
    #print("i",i)
    #print("meta",len(metadata_documents))
    #print ('header=',result[0])
    return metadata_documents

In [6]:
documents=[]
for url in urls:
    docs =[]
    docs=load_headers_creat_metadata(url)
    for doc in docs:
        documents.append(doc)
        #print(doc)
        #print("--------")
    
print(type(documents), len(documents))
print(documents[10].metadata)
print("-------------------------")
#print(documents[10].page_content)

<class 'list'> 1609
{'source': 'https://weclouddata.com/courses/online/business-intelligence-bootcamp', 'Title': 'Business Intelligence Bootcamp - WeCloudData', 'Header': 'About the Program', 'Section number': 10}
-------------------------


In [7]:

OnDcPtIns=["Adam Xing","Tanya Zhou","Tianshu Luan","Shaohua Zhang","Ken Ding", "Rogelio Cuevas"]
OnDePtIns=["Indrani Gorti","Zheng Xu","Edwin Guo","Stan Taov"]
i=0
for txtt in documents:
    #tx="Weekly Schedule"
    #tx="Edwin"
    #tx="instructors"
    tx="instructors & TA"
    
    if tx in txtt.page_content:
           print(txtt)
           print("<<<------------------------->>>")
           #print(doc.metadata)
           #print(doc.page_content)
           i=i+1
print(i)


Document(page_content=' >>>>>>\n URL :: https://weclouddata.com/courses/online/business-intelligence-bootcamp \n------\n Title :: Business Intelligence Bootcamp - WeCloudData \n-------\n Link :: /courses/online/business-intelligence-bootcamp \n------\n Section 19 :: Learn from the best instructors & TA >>>>>>>>\n content ::::  Learn from the best instructors & TA We’ve brought together a team of highly skilled and experienced instructors to help you learn effectively. Our instructors have a passion for teaching and a wealth of real-world experiences in their respective fields, so you can be confident that you’re learning from the best. Elvis Liang Business System Analyst, The Stars Group Elvis has Bachelor’s and Master’s degrees in Chemical Engineering. After in-depth exploration on career path of chemical engineering, Elvis found his passion in business and data analysis, and successfully switched his career to an online gaming industry working as a Business System Analyst at PokerSta

In [8]:
import nltk
from rake_nltk import Rake

# Download the necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

def find_keywords(question, num_keywords=5):
    r = Rake()
    r.extract_keywords_from_text(question)
    keywords = r.get_ranked_phrases()[:num_keywords]
    return keywords



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
query="Who are instructors for data engineering bootcamp?"
keywords = find_keywords(query)
print(keywords)

['data engineering bootcamp', 'instructors']


In [10]:
docs = documents
    
for keyword in keywords:
    
    relatedDocs = []
    for doc in docs:
        if keyword.lower() in doc.page_content.lower():
            relatedDocs.append(doc)
    docs=relatedDocs
    
if not relatedDocs:
    relatedDocs = documents  # Fallback to all documents if no relevant documents found
    
context = "\n\n\n".join([doc.page_content for doc in relatedDocs])
print(len(docs),context)



10  >>>>>>
 URL :: https://weclouddata.com/courses/online/data-engineering-bootcamp-part-time/ 
------
 Title :: Data Engineering Bootcamp - WeCloudData 
-------
 Link :: /courses/online/data-engineering-bootcamp-part-time/ 
------
 Section 9 :: Data Engineering >>>>>>>>>
 content ::::  Data Engineering Bootcamp Applied Data Engineering and Architect If you are looking for a learning program to help you become a data engineer, your search can stop here. WeCloudData’s Data Engineering Bootcamp is the most comprehensive and effective program to help you achieve that goal. Data Engineering programs aren’t easy to design because curriculums must be frequently updated due to the rapid changes in tools and platforms in the data landscape. Traditional university programs will not be able to keep it up to date. Data Engineering is also hard to teach because you need instructors with a strong industry background and hands-on experience. That’s why you don’t find many options out there. At WeClo

In [11]:
# Define the base template
template = """You are an intelligent chatbot.
Provide a detailed response to the question based only on the provided context.
Refrain from inventing any information not explicitly stated in the context.
If uncertain, simply state that you do not know instead of speculating.
If you do not find the answer in the context, please state 'I cannot find the answer from the uploaded file.'

{context}

Question: {question}
Answer:"""




In [12]:
# Find context based on keywords
def find_related_context(query, documents):
    keywords = find_keywords(query)
    
    docs = documents
    
    for keyword in keywords:
    
        relatedDocs = []
        for doc in docs:
            if keyword.lower() in doc.page_content.lower():
               relatedDocs.append(doc)
        docs=relatedDocs
    
    if not relatedDocs:
       relatedDocs = documents  # Fallback to all documents if no relevant documents found
    
    context = "\n\n".join([doc.page_content for doc in relatedDocs])
    #print(len(docs),context
    return context


def final_prompt(query, documents):
    final_context = find_related_context(query, documents)
    prompt_template = PromptTemplate(input_variables=["context", "question"], template=template)
    fprompt = prompt_template.format(context=final_context, question=query)
    return fprompt



In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
chunks = text_splitter.split_documents(documents)

# Check the length of docs
print(len(chunks))


# Display the first split document
print(chunks[22])

#print(chunks[22].metadata)
#print(chunks[22].page_content)


2021
page_content='content ::::  Drive Success with Interactive Learning Experience Daily Schedule 9:00am-5:00pm (Eastern) ** Real Project-based Learning * Learners who are also enrolled in the real client project program will spend the weekdays on project-based learning. Students will go through project on-boarding quizzes/training before getting assigned to client projects. Inquire today to learn more. 6:00pm-7:00pm (Eastern) TA Office Hour We host daily TA office hour before the lecture for the BI program so students can ask lecture/exercise related questions before the class. 7:00pm-10:00pm (Eastern) Lecture During the lectures, students will interact directly with the instructors via lessons, Q&As, and lecture demos. Weekly Schedule Monday Lecture (7-10pm ET) Lesson #1 of the week. Learners join live zoom sessions to learn new topics of the week. The instructor will spend 2 hours on theory, use cases and 1 hour on hands-on demo. Students are encouraged to follow along and ask a lo

In [14]:

# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-L6-v2"
#modelPath = "sentence-transformers/all-mpnet-base-v2"
#modelPath = "sentence-transformers/clips/mfaq"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device': 'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to True
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)


c:\Users\user\.conda\envs\mllrag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\user\.conda\envs\mllrag\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
persist_directory="./courses.db"
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_directory
)

db = vectordb
db._collection.count()

17870

In [29]:
# 'filter': {'Header': 'Learn from the best instructors & TA'}

k= 4 #5,..,10
query = "Who are instructors of Data Engineering Bootcamp?"
#query="Who is Indrani Gorti in data engineering bootcamp part time?"

print("------------similarity----------------")
#result=db.as_retriever(search_kwargs={"k": k}).invoke(query)
#result=db.as_retriever(search_kwargs={"k": k}, search_type = 'similarity').invoke(query)
#result1= db.as_retriever(search_kwargs={'k': k,'filter': {'Title': 'Data Engineering Bootcamp - WeCloudData'}}, search_type = 'similarity').invoke(query)
result1= db.as_retriever(search_kwargs={'k': k,'filter': {'Header': 'Learn from the best instructors & TA'}}, search_type = 'similarity').invoke(query)
#result1 = db.as_retriever(search_kwargs={'k': k, 'filter': {'Title': 'Data Engineering Bootcamp - WeCloudData', 'Header': 'Learn from the best instructors & TA'}}, search_type='similarity').invoke(query)

print("--Question   :  ",query)
print("--Response   :  ",result1)

print(type(result1))
for i in range(0,k):
    print(result1[i].metadata)

print("------------mmr----------------")
#result2= db.as_retriever(search_kwargs={'k': k,'filter': {'Title': 'Data Engineering Bootcamp - WeCloudData'}}, search_type = 'mmr').invoke(query)
result2= db.as_retriever(search_kwargs={'k': k,'filter': {'Header': 'Learn from the best instructors & TA'}}, search_type = 'mmr').invoke(query)
print("--Question   :  ",query)
print("--Response   :  ",result2)

print(type(result2))
for i in range(0,k):
    print(result2[i].metadata)


------------similarity----------------
--Question   :   Who are instructors of Data Engineering Bootcamp?
--Response   :   [Document(page_content='>>>>>>\n URL :: https://weclouddata.com/courses/online/data-science-bootcamp/ \n------\n Title :: Data Science Bootcamp - WeCloudData \n-------\n Link :: /courses/online/data-science-bootcamp/ \n------\n Section 19 :: Learn from the best instructors & TA >>>>>>>>', metadata={'Header': 'Learn from the best instructors & TA', 'Section number': 19, 'Title': 'Data Science Bootcamp - WeCloudData', 'source': 'https://weclouddata.com/courses/online/data-science-bootcamp/'}), Document(page_content='>>>>>>\n URL :: https://weclouddata.com/courses/online/data-science-bootcamp/ \n------\n Title :: Data Science Bootcamp - WeCloudData \n-------\n Link :: /courses/online/data-science-bootcamp/ \n------\n Section 19 :: Learn from the best instructors & TA >>>>>>>>', metadata={'Header': 'Learn from the best instructors & TA', 'Section number': 19, 'Title': 

In [16]:
# Define the base template
template = """You are an intelligent chatbot.
Provide a detailed response to the question based only on the provided context.
Refrain from inventing any information not explicitly stated in the context.
If uncertain, simply state that you do not know instead of speculating.
If you do not find the answer in the context, please state 'I cannot find the answer from the uploaded file.'

{context}

Question: {question}
Answer:"""

# Define the QA chain prompt template
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=['question', 'context'],
    template="I want to go over the  {context} and gnerate the answer for the {question}. I want answer with details and explanations, and the result should be exactly as the arange items."
)


In [17]:
query="Who are instructors of Data Engineering Bootcamp?"
#query="Who are instructors for data Science bootcamp?"
#query="Who are instructors for machine learning engineering bootcamp?"

#query="What is the weekly schedule for the Data Science Bootcamp?"
#query="What is the weekly schedule for the Data engineering Bootcamp?"
#query="What is the weekly schedule for the Machine Learning Engineering Bootcamp?"



keywords = find_keywords(query)
print(keywords)

['data engineering bootcamp', 'instructors']


In [18]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = mistral_llm
final_context = find_related_context(query, documents)

onedocs_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, output_key="query_doc")

result = onedocs_chain.run(question=query, context=final_context)
print(type(result))
print(result)

c:\Users\user\.conda\envs\mllrag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\user\.conda\envs\mllrag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<class 'str'>
I want to go over the   >>>>>>
 URL :: https://weclouddata.com/courses/online/data-engineering-bootcamp-part-time/ 
------
 Title :: Data Engineering Bootcamp - WeCloudData 
-------
 Link :: /courses/online/data-engineering-bootcamp-part-time/ 
------
 Section 9 :: Data Engineering >>>>>>>>>
 content ::::  Data Engineering Bootcamp Applied Data Engineering and Architect If you are looking for a learning program to help you become a data engineer, your search can stop here. WeCloudData’s Data Engineering Bootcamp is the most comprehensive and effective program to help you achieve that goal. Data Engineering programs aren’t easy to design because curriculums must be frequently updated due to the rapid changes in tools and platforms in the data landscape. Traditional university programs will not be able to keep it up to date. Data Engineering is also hard to teach because you need instructors with a strong industry background and hands-on experience. That’s why you don’t fin

In [20]:

llm = mistral_llm
final_context = find_related_context(query, documents)
prompt_template_2inputs = PromptTemplate(
    input_variables=['question', 'context'],
    template="I want to go over the  {context} and gnerate the answer for the {question}. I want answer with details and explanations, and the result should be exactly as the arange items."
)

# Setup RetrievalQA chain
qa_sim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 7}, search_type='similarity'),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template_2inputs}
)

#result1 = qa_Rm.run(question=query, context=adjusted_context)
result = qa_sim({"query": query})
response = result['result']  # The chatbot's response
source_documents = result['source_documents']  # The source documents used
prompt = prompt_template_2inputs

template_string = template
check_template = ChatPromptTemplate.from_template(template_string)

  
# Format the template string
formatted_template = template.format(
        question=query,
        prompt=QA_CHAIN_PROMPT,
        context=final_context
    )

response = llm(formatted_template)
    
# Get answers to the questions
print("<<<------------qa_sim---------------->>>")

print("\nQuestion---->>>",response.split("Question:")[1])

#print (response)

<<<------------qa_sim---------------->>>

Question---->>>  Who are instructors of Data Engineering Bootcamp?
Answer: The instructors for the Data Engineering Bootcamp are not explicitly mentioned in the provided context. However, some of the instructors for other courses offered by WeCloudData are listed, including Stan Taov, Indrani Gorti, Zheng Xu, and Edwin Guo. It's possible that some of these instructors may also teach in the Data Engineering Bootcamp, but the context does not confirm this.


1..mmr

In [21]:
# search_type = 'mmr'

llm = mistral_llm
final_context = find_related_context(query, documents)
prompt_template_3inputs = PromptTemplate(
    input_variables=['question','result', 'context'],
    template="I want to go over the  {context} and {source_documents} gnerate the useful answer for the {question}. I want answer with details and explanations, and the result should be exactly as the arange items."
)
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents
#mmr2_retriever= db.as_retriever(search_kwargs={'k': 4,'filter': {'Header': 'Drive Success with Interactive Learning Experience'}}, search_type = 'mmr')
mmr2_H_retriever= db.as_retriever(search_kwargs={'k': 4,'filter': {'Header': 'Learn from the best instructors & TA'}}, search_type = 'mmr')
#sim_retriever = db.as_retriever(search_kwargs={"k": 4}, search_type = 'similarity')
mmr_retriever = db.as_retriever(search_kwargs={"k": 4}, search_type = 'mmr')


qa_Rm = RetrievalQA.from_chain_type(llm,
                                chain_type='stuff',
                                retriever=mmr2_H_retriever,
                                return_source_documents=True
                                )


#result1 = qa_Rm.run(question=query, context=adjusted_context)
result1 = qa_Rm({"query": query})
response = result1['result']  # The chatbot's response
source_documents = result1['source_documents']  # The source documents used
prompt = prompt_template_3inputs

template_string = template
check_template = ChatPromptTemplate.from_template(template_string)

  
# Format the template string
formatted_template = template.format(
        question=query,
        prompt=prompt_template_3inputs,
        context=final_context,
        source_documents=source_documents
    )

resp = llm(formatted_template)
    
# Get answers to the questions
print("<<<------------qa_mmr---------------->>>")

#respoo= result1["result"].split("Question:")[1]
print("\nQuestion---->>>",resp.split("Question:")[1])
#print(respoo)
#print(adjusted_context)

<<<------------qa_mmr---------------->>>

Question---->>>  Who are instructors of Data Engineering Bootcamp?
Answer: The instructors for the Data Engineering Bootcamp are not explicitly mentioned in the provided context. However, some of the instructors for other courses offered by WeCloudData are listed, including Stan Taov, Indrani Gorti, Zheng Xu, and Edwin Guo. It's possible that some of these instructors may also teach in the Data Engineering Bootcamp, but the context does not confirm this.


### 2..MultiQuery Retriever

In [22]:
from langchain.retrievers import MultiQueryRetriever
from langchain.chains import create_retrieval_chain

document_chain = create_stuff_documents_chain(mistral_llm, QA_CHAIN_PROMPT)

#db_retrieval_chain = create_retrieval_chain(db_retriever, document_chain)

#sim_retriever=db.as_retriever(search_kwargs={"k": 3}, search_type = 'similarity')
#mmr_retriever=db.as_retriever(search_kwargs={"k": 3}, search_type = 'mmr')

MQR_retriever = MultiQueryRetriever.from_llm(retriever=mmr_retriever, llm=llm)

MQR_retrieval_chain = create_retrieval_chain(MQR_retriever, document_chain)
#response = retrieval_chain.invoke({"question": "how can langsmith help with testing?"})
#response = retrieval_chain.invoke({"question": "How can langsmith help with testing?", "context": "relevant text or background information"})
#print(response["answer"])

In [23]:
llm = mistral_llm
qa_MQR = RetrievalQA.from_chain_type(llm,
                                chain_type='stuff',
                                retriever=MQR_retriever,
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


#result1 = qa_Rm.run(question=query, context=adjusted_context)
result1 = qa_MQR({"query": query})
response = result1['result']  # The chatbot's response
source_documents = result1['source_documents']  # The source documents used
prompt = prompt_template_3inputs

template_string = template
check_template = ChatPromptTemplate.from_template(template_string)

  
# Format the template string
formatted_template = template.format(
        question=query,
        prompt=prompt_template_3inputs,
        context=final_context,
        source_documents=source_documents
    )

resp = llm(formatted_template)
    
# Get answers to the questions
print("<<<------------qa_MQR---------------->>>")

#respoo= result1["result"].split("Question:")[1]
print("\nQuestion---->>>",resp.split("Question:")[1])

<<<------------qa_MQR---------------->>>

Question---->>>  Who are instructors of Data Engineering Bootcamp?
Answer: The instructors for the Data Engineering Bootcamp are not explicitly mentioned in the provided context. However, some of the instructors for other courses offered by WeCloudData are listed, including Stan Taov, Indrani Gorti, Zheng Xu, and Edwin Guo. It's possible that some of these instructors may also teach in the Data Engineering Bootcamp, but the context does not confirm this.


#### 3.. Contextual Compression Retriever

In [25]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

sim_retriever = db.as_retriever(search_kwargs={"k": 4}, search_type = 'similarity')
#mmr_retriever = db.as_retriever(search_kwargs={"k": 4}, search_type = 'mmr')

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=sim_retriever)

compressed_docs = compression_retriever.get_relevant_documents(query=query)
#print(compressed_docs)


In [26]:
llm = mistral_llm
qa_COMP = RetrievalQA.from_chain_type(llm,
                                chain_type='stuff',
                                retriever= compression_retriever,
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


#result1 = qa_Rm.run(question=query, context=adjusted_context)
result1 = qa_COMP({"query": query})
response = result1['result']  # The chatbot's response
source_documents = result1['source_documents']  # The source documents used
prompt = prompt_template_3inputs

template_string = template
check_template = ChatPromptTemplate.from_template(template_string)

  
# Format the template string
formatted_template = template.format(
        question=query,
        prompt=prompt_template_3inputs,
        context=final_context,
        source_documents=source_documents
    )

resp = llm(formatted_template)
    
# Get answers to the questions
print("<<<------------COMP---------------->>>")

#respoo= result1["result"].split("Question:")[1]
print("\nQuestion---->>>",resp.split("Question:")[1])
#print(respoo)
#print(adjusted_context)

<<<------------COMP---------------->>>

Question---->>>  Who are instructors of Data Engineering Bootcamp?
Answer: The instructors for the Data Engineering Bootcamp are not explicitly mentioned in the provided context. However, some of the instructors for other courses offered by WeCloudData are listed, including Stan Taov, Indrani Gorti, Zheng Xu, and Edwin Guo. It's possible that some of these instructors may also teach in the Data Engineering Bootcamp, but the context does not confirm this.


### 4..Ensemble Retriever

In [27]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever


bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 2

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, mmr_retriever], weights=[0.5, 0.5]
)

docs = ensemble_retriever.get_relevant_documents(query=query)
#docs
qa_ENSEM = RetrievalQA.from_chain_type(llm,
                                chain_type='stuff',
                                retriever= ensemble_retriever,
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


#result1 = qa_Rm.run(question=query, context=adjusted_context)
result1 = qa_ENSEM({"query": query})
response = result1['result']  # The chatbot's response
source_documents = result1['source_documents']  # The source documents used
prompt = prompt_template_3inputs

template_string = template
check_template = ChatPromptTemplate.from_template(template_string)

  
# Format the template string
formatted_template = template.format(
        question=query,
        prompt=prompt_template_3inputs,
        context=final_context,
        source_documents=source_documents
    )

resp = llm(formatted_template)
    
# Get answers to the questions
print("<<<------------ENSEMBEL---------------->>>")

#respoo= result1["result"].split("Question:")[1]
print("\nQuestion---->>>",resp.split("Question:")[1])


<<<------------ENSEMBEL---------------->>>

Question---->>>  Who are instructors of Data Engineering Bootcamp?
Answer: The instructors for the Data Engineering Bootcamp are not explicitly mentioned in the provided context. However, some of the instructors for other courses offered by WeCloudData are listed, including Stan Taov, Indrani Gorti, Zheng Xu, and Edwin Guo. It's possible that some of these instructors may also teach in the Data Engineering Bootcamp, but the context does not confirm this.
